In [3]:
#Import libraries
import os
import glob
import cv2
import random
import tensorflow as tf
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras as kr
import tensorflow_hub as hub
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, recall_score, matthews_corrcoef, confusion_matrix, auc, roc_curve
from sklearn import svm
from sklearn.model_selection import train_test_split  
from sklearn.metrics import make_scorer

In [ ]:
#Read input images and assign labels based on folder names
print(os.listdir("data/")) #The dataset is located in a folder called data
SIZE = 250  #Resize images

In [ ]:
#Capture training data and labels into respective lists
images = []
labels = [] 

for directory_path in glob.glob("data/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        images.append(img)
        labels.append(label)

#Convert lists to arrays        
images = np.array(images)
labels = np.array(labels)

In [ ]:
Seed_0 = random.randint(1,2**32 - 1) #Seed used for split
print(Seed_0)

In [49]:
# Split dataset in train (0.8) and test (0.2)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state = Seed_0)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_test)
test_labels_encoded = le.transform(y_test)
le.fit(y_train)
train_labels_encoded = le.transform(y_train)

#print(test_labels_encoded)
print(train_labels_encoded)

len(train_labels_encoded)
Y_train = train_labels_encoded
Y_test = test_labels_encoded
#0 = Aprovechable
#1 = No_Aprovechable

In [7]:
# Import Inception V1 and freeze all layers
Inception = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v1/classification/5", 
           trainable=False, 
           arguments=dict(batch_norm_momentum=0.997))

Inception.build([None, 250, 250, 3])

In [ ]:
# Create a new model and use InceptionV1 with fine-tuning method
LR = 0.001 #Learning rate

model = tf.keras.Sequential([
    
    tf.keras.layers.Rescaling( 1./255, input_shape = ( 250, 250, 3 ) ), #Normalize data
    Inception, #Import InceptionModel
    tf.keras.layers.Dense( units = 256, activation = 'relu' ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense( units = 1, activation = 'sigmoid' )
])

model.summary() #Summary of the architecture

# Compile model with Adam as optimizer and BinaryCrossEntropy as loss function
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate = LR ),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy', 'AUC', tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])

# Train model with callbacks
N_epochs = 20
history = model.fit( x = X_train, y = Y_train, epochs = N_epochs, shuffle = True )

In [ ]:
#Make predictions in the test split
Predictions = model.predict(X_test)

In [ ]:
#Convert predictions in integers 
Y_predict = []
for i in Predictions:
  if i > 0.5:
    Y_predict.append(1)
  else:
    Y_predict.append(0)

#Calculate and show metrics
tn, fp, fn, tp = confusion_matrix(y_true=Y_test, y_pred=Y_predict).ravel()
specificity_ = tn / (tn+fp)
acc_ = accuracy_score( y_true = Y_test, y_pred = Y_predict )
f1_ = f1_score( y_true = Y_test, y_pred = Y_predict )
fpr_auc, tpr_auc, thresholds_keras = roc_curve(Y_test, Y_predict)
auc_ = auc(fpr_auc, tpr_auc)
recall_ = recall_score( y_true = Y_test, y_pred = Y_predict )
mcc_ = matthews_corrcoef( y_true = Y_test, y_pred = Y_predict )
print("Accuracy = {acc}".format(acc=acc_))
print("F1 score = {f1}".format(f1=f1_))
print("MCC = {mcc}".format(mcc=mcc_))
print("AUC = {auc}".format(auc=auc_))
print("Recall = {rc}".format(rc=recall_))
print("Specificity = {sp}".format(sp=specificity_))
print("Seed = {seed}".format(seed=Seed_0))

In [ ]:
#Plot ROC curve
plt.figure("ROC-AUC Curve")
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_auc, tpr_auc, label='InceptionV1 (area = {:.3f})'.format(auc_), color = 'tab:orange')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#Ploting Validation metrics
xdata = [ x for x in range(N_epochs) ]
fig1 = plt.figure('Metrics')
plt.plot(xdata, recall_, color = 'c', linewidth = 2,linestyle='dashed')
plt.plot(xdata, specificity_, color = 'm',linewidth = 2,linestyle='dashed')
plt.plot(xdata, mcc_, color = 'b',linewidth = 2.5)
plt.plot(xdata, f1_, color = 'r',linewidth = 2.5)
plt.plot(xdata, history.history['val_auc'], color = 'tab:orange',linewidth = 2.5)
plt.plot(xdata, history.history['val_accuracy'], color = 'tab:olive',linewidth = 2.5)
plt.legend(['Recall','Specificity','MCC','F1','AUC','Accuracy'])
plt.title('Validation metrics')
plt.xlabel('Epochs')
plt.ylim(0, 1)

In [ ]:
#Ploting accuracy metrics Training vs Validation
fig2 = plt.figure('Accuracy')
plt.plot(xdata, history.history['accuracy'], color = 'tab:green',linewidth = 2.5)
plt.plot(xdata, history.history['val_accuracy'], color = 'tab:olive',linewidth = 2.5)
plt.legend(['Accuracy','val_accuracy'])
plt.title('Accuracy vs val_accuracy')
plt.xlabel('Epochs')
plt.ylim(0, 1)
plt.show()